# Build a WFlow model anywhere

In this notebook we will setup a WFlow model coupled to the FloodAdapt backend

## **Step 0:** Import dependencies

In [15]:
from pathlib import Path
import hydromt
import geopandas as gpd
from hydromt.log import setuplog
from hydromt.config import configread
from hydromt_wflow import WflowModel

In [10]:
model_name = 'Humber'
model_path = Path('c:/Repositories/DT-flood/FloodAdapt_database') / model_name
# model_path = Path("../FloodAdapt_database") / model_name

wf_root = model_path / Path('static/templates/wflow')

wf_logger_name = 'WFlow_Logger'

region_fn = model_path / 'HumberDelta_large.geojson'

In [11]:
# Use this for own data_catalog file
data_catalog_fn = Path('c:/Users/tromp_wm/OneDrive - Stichting Deltares/Documents/InterTwin Project/Data/DTFlood_test_catalog.yml')

data_dict = {
    'topo': 'copdem30',
    'bathy': 'gebco',
    'waterlevels': 'gtsm_hourly',
    'meteo': 'era5_hourly',
    'infiltration': 'gcn250',
    'lulc': 'esa_worldcover',
    'basins': 'BasinAtlas',
}

In [12]:
wf_logger = setuplog(wf_logger_name,log_level=10)

# data_catalog = hydromt.DataCatalog(data_libs=data_catalog_fn,logger=wf_logger)

wf = WflowModel(data_libs=[data_catalog_fn, 'deltares_data=v2022.7'], root=wf_root, mode='w+')

2024-01-29 09:48:45,923 - WFlow_Logger - log - INFO - HydroMT version: 0.8.0


In [17]:
ini_fn = model_path / "wflow_build.ini"
opt = configread(ini_fn)

region = gpd.read_file(region_fn)

In [18]:
wf.build(region={'basin': region, 'outlets': True}, opt=opt)

No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata
No numerical nodata value found, skipping set_nodata


[########################################] | 100% Completed | 9.11 sms
